<div>
    <img style="float:left;" src="/home/jzadorozhnaya/Supercomputers_msu/7_sem/Report/vmk.png" width="12%">
    <img style="float:right;margin-top:0px;" src="/home/jzadorozhnaya/Supercomputers_msu/7_sem/Report/msu.png" width="12%">
    <div style="margin-top:25px;text-align:center;font-size:25px">МОСКОВСКИЙ ГОСУДАРСТВЕННЫЙ УНИВЕРСИТЕТ</div>
    <div style="margin-top:35px;text-align:center;font-size:18px">имени М.В. Ломоносова</div>
    <div style="margin-top:25px;text-align:center;font-size:25px">Факультет вычислительной математики и кибернетики</div>
    <br><br><br><br><br><br><br><br>
<div>

<strong><div style="text-align:center;font-size:40px">Практикум по курсу</div>
<strong><div style="margin-top:30px;text-align:center;font-size:30px">"Распределённые системы"</div>
<strong><div style="margin-top:55px;text-align:center;font-size:30px">Алгоритм ALL_REDUCE для транспьютерной матрицы</div>
<strong><div style="margin-top:20px;text-align:center;font-size:30px">Разработка отказоустойчивой версии программы для задачи Gauss</div>
<strong><div style="margin-top:55px;text-align:center;font-size:30px">ОТЧЕТ</div>
<strong><div style="margin-top:25px;text-align:center;font-size:30px">о выполненном задании</div>
<div style="margin-top:55px;text-align:center;font-size:20px">студентки 421 учебной группы факультета ВМК МГУ</div>
<div style="margin-top:25px;text-align:center;font-size:20px">Задорожной Юлии Андреевны</div>
<div style="margin-top:100px;text-align:center;font-size:20px">Москва, 2022 г.</div>

# Часть 1: Постановка задачи.

### Задача 1: Алгоритм ALL_REDUCE для транпьютерной матрицы

Требуется разработать и реализовать:

В транспьютерной матрице размером 5*5, в каждом узле которой находится один процесс,
необходимо выполнить операцию нахождения максимума среди 25 чисел (каждый процесс
имеет свое число). Найденное максимальное значение должно быть получено на каждом про-
цессе.

Реализовать программу, моделирующую выполнение операции MPI_ALLREDUCE на транспью-
терной матрице при помощи пересылок MPI типа точка-точка.

Оценить сколько времени потребуется для выполнения операции MPI_ALLREDUCE, если все про-
цессы выдали эту операцию редукции одновременно. Время старта равно 100, время передачи
байта равно 1 (Ts=100,Tb=1). Процессорные операции, включая чтение из памяти и запись в
память, считаются бесконечно быстрыми.

### Задача 2: Разработка отказоустойчивой версии программы для задачи Gauss

Доработать MPI-программу, реализованную в рамках курса “Суперкомпьютеры и параллельная обработка данных”. 
Добавить контрольные точки для продолжения работы программы в случае сбоя. Реализовать один из 3-х сценариев работы после сбоя: 

a) продолжить работу программы только на “исправных” процессах; 

б) вместо процессов, вышедших из строя, создать новые MPI-процессы, которые необходимо использовать для продолжения расчетов; 

**в) при запуске программы на счет сразу запустить некоторое дополнительное количество MPI-процессов, которые использовать в случае сбоя.**

# Часть 2: Описание алгоритма.


### Задача 1: Алгоритм ALL_REDUCE для транпьютерной матрицы.

В транспьютерной матрице размером 5*5, в каждом узле которой находится один процесс, необходимо выполнить операцию нахождения максимума среди 25 чисел (каждый процесс имеет свое число). Найденное максимальное значение должно быть получено на процессе с координатами (0,0) и затем передано другим процессам.

Минимальное время оценивается через минимальное расстояние между двумя самыми дальними процессами в матрице. 

Данную задачу разложим на две части: логика задачи MPI_Gather + MPI_Scatter

Применим такой алгоритм:

1) Поскольку передачи в транспьютерной матрице возможны только к соседним узлам, то будем передавать число каждого процесса снизу вверх, производя сразу операцию редукции (вычисление максимума). Посмотреть процесс передачи "cнизу-вверху" можно посмотреть ниже:

[jupyter](7_sem/Report/inp.png)


2) Теперь смотрим на процессы в верхней строчке: в этих процессах также происходит операция вычисление максимума с имеющимися у них числами (это нечетные цифры на рисунке, кроме 1), последовательная передача полученного числа в соседний узел (это четные цифры на рисунке), снова вычисление максимума... и так, пока не окажемся с максимальным числом среди 25 элементов транспьютерной матрицы на процессе (0, 0).

<img style="float:center;" src="/home/jzadorozhnaya/Supercomputers_msu/7_sem/Report/inp2.png" width="55%">

3. Теперь максимальное число среди всех 25 процессов лежит в узле (0, 0). Его необходимо ~~сообщить~~ передать всем остальным процессам.
Сначала из процесса (0, 0) мы передаем, например, всем процессам, лежащим в нулевой строке (красная стрелка), а затем из нулевой строки передаем это число всем процессам, лежащим под нулевой строкой (черная стрелка). Буквально, это операция, обратная операции передачи значения в процесс (0, 0), только без операции редукции. 

Визуально это будет выглядеть так: 

<img style="float:center;" src="/home/jzadorozhnaya/Supercomputers_msu/7_sem/Report/inp4.png" width="55%">



Данный алгоритм был реализован с помощью функций MPI_Isend и MPI_Recv. Получение топологии в виде транспьютерной матрицы произведено с помощью функции MPI_Cart_rank.

Оценим время работы алгоритма. Если время старта равно 100, время передачи байта равно 1 (Ts=100,Tb=1), то время выполнения операции передачи в одну сторону рассчитывается следующим образом:

```
                                 time = num_steps · (Ts + n * Tb)
```

где n - размер передаваемого сообщения в байтах. 


В нашем случае сообщением является число, размер которого может быть равен, например, 4 байтам (integer).
Таким образом, при n = 5, и при двух проходах (сначала передача в (0, 0), а затем такая же передача уже в обратную сторону):


In [1]:
def calculate_time(size, ts = 100, tb = 1, count = 4):
    return (size-1) * 4 * (ts + tb * count)

print(calculate_time(5, ts = 100, tb = 1, count = 4))


1664


Суммарное время: 1664 mc

### Задача 2: Разработка отказоустойчивой версии программы для задачи Gauss.


Для реализации отказоустойчивой версии был выбран метод, который создает дополнительные процессы, чтобы в случае ошибки заменить ими
“испорченные” процессы и продолжить вычисления тем же количеством процессов. Другими словами, в новой программе происходит расставлении контрольных точек и в случае ошибки осуществляется возврат к этим контрольным точкам, удаление "испорченных" процессов, создание новых и последующая синхронизация и вычисления с тем же количеством процессов.

Для этого рассмотрим основные функции программы: 


1) ``` verbose_errhandler() ```

Данная функция описывает реакцию процессов в случае сбоя. В результате
сбоя каждый процесс должен обновить свою рабочую группу (удалить из неё
мёртвый процесс с помощью MPIX_Comm_shrink) и после этого сделать loadData,
чтобы иметь правильные данные на момент начала чекпоинта.

2. ```loadData()```

В случае сбоя, данные загружаются из файлов в память каждого процесса. Это сделано для того, чтобы никакой процесс не имел на каком-то шаге
наполовину обновленную матрицу. Матрицы должны быть сброшены к предыдущему шагу. После загрузки процессы ждут друг друга с помощью MPI_Barrier.

3. ```saveData()```

Данная функция отвечает за запись данных в файлы. Так как все процессы имеют одинаковые данные, то записью занимается процесс с рангом 0.
Остальные же процессы ждут завершения процесса записи с помощью MPI_Barrier.

4. Организация ~~чекпоинта~~ контрольной точки: 

```
    saveData();
    ...
    flag = true; //флаг для входа в цикл
    while( flag || err_fl){
        err_fl = false;
        flag = false;
        ...
        if(err_fl == false){
            MPI_BARRIER(comm)
    }
    saveData();
```

В случае если в данном блоке возникает ошибка, то флаг err_fl будет true. Тогда блок будет выполнен ещё раз с данными, которые были на последнем чекпоинте (при последнем saveData). Если блок будет завершён успешно всеми процессами, то идёт обновление бинарного файл(checkpoint).

# Часть 3: Способы запуска программы.

### Задача 1: Алгоритм ALL_REDUCE для транпьютерной матрицы

Код, прилагаемый в 6 части, необходимо запустить c помощью bash-script файла start.sh. 

Это можно сделать с помощью команды ./start.sh <количество процессов>

Содержимое файла start.sh:

``` bash
    mpicc main.c -o test -lm
    mpiexec -n $1 ./test
```

### Задача 2: Разработка отказоустойчивой версии программы для задачи Gauss

Для начала нужно запустить docker container. 

Директория Task_2 cодержит реализацию отказоустойчивой программы Gauss (main.c), Makefile, а также три скрипта, реализующие запуск докера,  сборку и запуск программы. Обязательно требуется проделать первую команду:

`./start_docker.sh` - будет извлекать все необходимые библиотеки из среды

Содержимое `start_docker.sh`:

``` bash
    ulfm_image=abouteiller/mpi-ft-ulfm
    docker pull $ulfm_image
```



`./build.sh` - сборка проекта

Содержимое `build.sh`:

``` bash
    sudo docker run -v $PWD:/sandbox:Z abouteiller/mpi-ft-ulfm make
```



`./qstart.sh` - начать тест с предварительно написанными параметрами, такими как номер базового процесса = 5, размер матрицы 10X10

Содержимое `qstart.sh`:

``` bash
    sudo docker run -v $PWD:/sandbox:Z abouteiller/mpi-ft-ulfm mpirun --oversubscribe -mca btl tcp,self -np 5 a.out 10
```




`./start.sh -np  "num_proc" a.out "matrix_size(one param)" "number of additional proc, default = 2"` - чтобы начать настраиваемый тест

Содержимое `start.sh`:
``` bash
    sudo docker run -v $PWD:/sandbox:Z abouteiller/mpi-ft-ulfm mpirun --oversubscribe -mca btl tcp,self "$@"
```

Содержимое `Makefile`:

``` makefile
	local_build:
		mpicc  -g main.c

	clean:
		rm -f ./*_res
		rm -f sec

```


`make local_build` - локальная сборка

`make clean` - удаление генератов

# Часть 4: Примеры запуска и выводов программы.

### Задача 1: Алгоритм ALL_REDUCE для транпьютерной матрицы

<img style="float:center;" src="/home/jzadorozhnaya/Supercomputers_msu/7_sem/Report/task1_ex.png" width="80%">

# Часть 5: Выводы.


# Часть 6: Приложение.

### Задача 1: Алгоритм ALL_REDUCE для транпьютерной матрицы

main.c:

``` c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

#include <mpi.h>
#include <unistd.h>

MPI_Request perform_operation(int x, int y) {
    return x > y ? x : y;
}

MPI_Request send_int(int x, int y, MPI_Comm comm, int *data) {
    int target_rank;
    int coords[2] = {x, y};
    MPI_Cart_rank(comm, coords, &target_rank);

    MPI_Request request;
    MPI_Isend(data, 1, MPI_INT, target_rank,
              0, MPI_COMM_WORLD, &request);
    return request;
}

int receive_int(int x, int y, MPI_Comm comm) {
    int target_rank;
    int coords[2] = {x, y};
    MPI_Cart_rank(comm, coords, &target_rank);

    int data;
    MPI_Recv(&data, 1, MPI_INT, target_rank,
             MPI_ANY_TAG, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    return data;
}

int main(int argc, char **argv) {
    int matrix_size;

    MPI_Init(&argc, &argv);

    MPI_Comm_size(MPI_COMM_WORLD, &matrix_size);

    matrix_size = (int) sqrt((double) matrix_size);

    int rank, size;

    int ndims = 2;

    int n_size[2] = {matrix_size, matrix_size};
    int periodic[2] = {0, 0};

    MPI_Comm cart_comm;

    MPI_Cart_create(MPI_COMM_WORLD, ndims, n_size, periodic, 1, &cart_comm);

    MPI_Comm_rank(cart_comm, &rank);
    MPI_Comm_size(cart_comm, &size);

    srand(time(NULL) + rank);

    int coords[2];
    MPI_Cart_coords(cart_comm, rank, ndims, coords);

    int my_number = rand() % 128;

    printf("My number is %4d, my coords is <%2d, %2d>\n", my_number, coords[0], coords[1]);
    sleep(1);

    int number;
    if (coords[0] == 0) {
        number = receive_int(coords[0] + 1, coords[1], cart_comm);
        my_number = perform_operation(my_number, number);
        printf("Got %d, my coords is <%2d, %2d>\n", number, coords[0], coords[1]);


        if (coords[1] + 1 != matrix_size) {
            number = receive_int(coords[0], coords[1] + 1, cart_comm);
            my_number = perform_operation(my_number, number);
            printf("Got %d, my coords is <%2d, %2d>\n", number, coords[0], coords[1]);
        }

        if (coords[1] != 0) {
            MPI_Request request;
            request = send_int(coords[0], coords[1] - 1, cart_comm, &my_number);
            MPI_Wait(&request, MPI_STATUS_IGNORE);
        }
    } else {
        if (coords[0] + 1 != matrix_size) {
            number = receive_int(coords[0] + 1, coords[1], cart_comm);
            my_number = perform_operation(my_number, number);

            printf("Got %d, my coords is <%2d, %2d>\n", number, coords[0], coords[1]);
        }

        MPI_Request request;
        request = send_int(coords[0] - 1, coords[1], cart_comm, &my_number);
        MPI_Wait(&request, MPI_STATUS_IGNORE);
    }

    sleep(1);

    if (coords[0] == 0 && coords[1] == 0) {
        printf("Total number is %d\n", my_number);
    }


    MPI_Barrier(cart_comm);
    sleep(1);

    if (coords[0] == 0) {
        if (coords[1] != 0) {
            my_number = receive_int(coords[0], coords[1] - 1, cart_comm);
            printf("Got number, my coords is <%2d, %2d>\n", coords[0], coords[1]);
        }

        MPI_Request r1, r2;
        if (coords[1] + 1 != matrix_size) {
            r1 = send_int(coords[0], coords[1] + 1, cart_comm, &my_number);
        }
        r2 = send_int(coords[0] + 1, coords[1], cart_comm, &my_number);
        if (coords[1] + 1 != matrix_size) {
            MPI_Wait(&r1, MPI_STATUS_IGNORE);
        }
        MPI_Wait(&r2, MPI_STATUS_IGNORE);
    } else {
        my_number = receive_int(coords[0] - 1, coords[1], cart_comm);
        printf("Got number, my coords is <%2d, %2d>\n", coords[0], coords[1]);
        MPI_Request r1;
        if (coords[0] + 1 != matrix_size) {
            r1 = send_int(coords[0] + 1, coords[1], cart_comm, &my_number);
            MPI_Wait(&r1, MPI_STATUS_IGNORE);
        }
    }

    MPI_Finalize();

    return 0;
}
```

### Задача 2: Разработка отказоустойчивой версии программы для задачи Gauss

main.c:

``` c
#include <math.h>
#include <stdlib.h>
#include <stdio.h>
#include <string.h>
//#include <stdio.h>
#include <sys/time.h>
#include <mpi.h>
#include "mpi-ext.h"
#include <signal.h>


void prt1a(char *t1, double *v, int n, char *t2);

void print_matrix(double *a);


MPI_Comm main_comm = MPI_COMM_WORLD;
int N ;
double *A;
#define A(i, j) A[(i) * (N + 1) + (j)]
double *X;
bool reverse_sub = false;
bool err_fl = false;
int proc_num, myrank;
//int errs = 0;
//bool was = false;

int additional_procs = 2;

//char** v;

//bool trouble_barrier = false;


static void saveData()
{
    if (myrank == 0) {
        FILE* f = fopen("elim_save.bin", "wb");
        fwrite(&A[0], sizeof(double),  (N-1)*N, f);
        fclose(f);

        if (reverse_sub) {
            FILE* f = fopen("rs_save.bin", "wb");
            fwrite(&X[0], sizeof(double),  N, f);
            fclose(f);
        }
    }
    //printf("here-\n");
    //fflush(stdout);
    //MPI_Barrier(main_comm);
    MPI_Barrier(main_comm);
}

static void loadData()
{

    
    //if(myrank < proc_num){
        FILE* f = fopen("elim_save.bin", "rb");
        fread(&A[0], sizeof(double), (N-1)*N, f);
        fclose(f);
        if (reverse_sub) {
            FILE* f = fopen("rs_save.bin", "wb");
            fwrite(&X[0], sizeof(double),  N, f);
            fclose(f);
        }
    //}    
    //printf("Proc %d\n", myrank);
    //fflush(stdout);
    MPI_Barrier(main_comm);
}


static void verbose_errhandler(MPI_Comm* pcomm, int* perr, ...)
{
    //errs++;
    MPI_Comm comm = *pcomm;
    int err = *perr;
    char errstr[MPI_MAX_ERROR_STRING];
    int i, rank, size, nf, len, eclass;
    MPI_Group group_c, group_f;
    int *ranks_gc, *ranks_gf;


    MPI_Error_class(err, &eclass);
    if( MPIX_ERR_PROC_FAILED != eclass ) {
        printf("Rank %d / %d: Notified of error %s. %d found dead: { ",
           rank, size, errstr, nf);
        MPI_Abort(comm, err);// stops all procesees if some other err occured
    }
    MPI_Comm_rank(comm, &rank);
    MPI_Comm_size(comm, &size);
    //was = true;
    MPIX_Comm_failure_ack(comm);
    MPIX_Comm_failure_get_acked(comm, &group_f);
    MPI_Group_size(group_f, &nf);
    MPI_Error_string(err, errstr, &len);
    // MPIX_Comm_revoke(comm);//some error occures if revoke is used


    printf("Rank %d / %d: Notified of error %s. %d found dead: { ",
           rank, size, errstr, nf);

    ranks_gf = (int*)malloc(nf * sizeof(int));
    ranks_gc = (int*)malloc(nf * sizeof(int));
    MPI_Comm_group(comm, &group_c);
    for(i = 0; i < nf; i++)
        ranks_gf[i] = i;
    MPI_Group_translate_ranks(group_f, nf, ranks_gf,
                              group_c, ranks_gc);
    for(i = 0; i < nf; i++)
        printf("%d ", ranks_gc[i]);
    printf("}\n");
    additional_procs -= nf;
    //MPIX_Comm_revoke(main_comm);
    MPIX_Comm_shrink(comm, &main_comm);// leave out of communicator dead processes
    MPI_Comm_rank(main_comm, &myrank);
    MPI_Comm_size(main_comm, &proc_num);
    
    proc_num -= additional_procs;
    loadData();
    //fflush(stdout);
    err_fl = true;
    free(ranks_gc);
    free(ranks_gf);
    //printf("%d\n", myrank);
    //fflush(stdout);
    MPI_Barrier(main_comm);


}

int main(int argc, char **argv)
{
    MPI_Init(&argc, &argv);
    //* adding extra processes to calculate on the same number of processes if error occures*
    
    if(argc == 3){ //enable custum number of additional proc
        additional_procs = atoi(argv[2]);
    }
    MPI_Comm parent_comm, intercom_comm;
    MPI_Comm_get_parent(&parent_comm);
    if (parent_comm == MPI_COMM_NULL) {
        MPI_Comm_spawn("./a.out", 
                argv + 1, 
                additional_procs, 
                MPI_INFO_NULL, 0, 
                main_comm, 
                &intercom_comm, 
                MPI_ERRCODES_IGNORE);
    } else {
        intercom_comm = parent_comm;
        //printf("helll%d", atoi(argv[1]));
        fflush(stdout);

    }
    MPI_Intercomm_merge(intercom_comm, (parent_comm == MPI_COMM_NULL ? 0 : 1), &main_comm);

    //*intializtion of some variables*
    MPI_Comm_size(main_comm, &proc_num);
    MPI_Comm_rank(main_comm, &myrank);
    proc_num = proc_num - additional_procs;
    fflush(stdout);
    MPI_Errhandler errh;
    MPI_Comm_create_errhandler(verbose_errhandler, &errh);
    MPI_Comm_set_errhandler(main_comm, errh);//error handler setup completed
    int i, j, k;
    bool flag = false;
    N = atoi(argv[1]);//matrix size

    fflush(stdout);
    /* create arrays */
    A = (double *)malloc(N * (N + 1) * sizeof(double));
    X = (double *)malloc(N * sizeof(double));
    if (myrank == 0) printf("GAUSS %dx%d\n----------------------------------\n", N, N);
    srand(12345);
    /* initialize array A*/
    for (i = 0; i <= N - 1; i++)
        for (j = 0; j <= N; j++)
            if (i == j || j == N)
                A(i, j) = 1.f;
            else
                A(i, j) = 0.f;

    double time0 = MPI_Wtime();
    saveData();
    // if(!trouble_barrier){
    // }else{
    //     trouble_barrier = false;
    // }

    /* elimination */

    
    //err_fl = false;
    if (myrank == 0){
        MPI_Barrier(main_comm);
        raise(SIGKILL);
    }else{
        MPI_Barrier(main_comm);
    }
    //bool was = false; 
    flag = true;
    while (err_fl || flag) {
        err_fl = false;
        
            for (i = 0; i < N - 1; i++)
            {
                //printf("myrank = %d\n", myrank);
                MPI_Bcast(&A(i, i + 1), N - i, MPI_DOUBLE, i % proc_num, main_comm);
                if(myrank >= proc_num){ continue;}
                for (k = myrank; k <= N - 1; k += proc_num) {
                    if (k < i + 1) {
                        continue;
                    }
                    for (j = i + 1; j <= N; j++) {
                        A(k, j) = A(k, j) - A(k, i) * A(i, j) / A(i, i);
                    }
                }
            }
        flag = false;
        if (!err_fl)
                //printf(" %d HERE4!!!!1 , %d\n", myrank, err_fl);
            MPI_Barrier(main_comm);

    }
    //printf(" %d HERE4!!!!1 , %d\n", myrank, err_fl);

    saveData();
    //printf(" %d HERE4!!!!1 , %d\n", myrank, err_fl);

    
    /* reverse substitution */
    //printf("here2\n");
    //err_fl = false;
    flag = true;
    while (err_fl || flag) {
        err_fl = false;
        X[N - 1] = A(N - 1, N) / A(N - 1, N - 1);
        flag = false;
        if (!err_fl)
            MPI_Barrier(main_comm);
        //err_fl = false;
    }
    saveData();
    reverse_sub = true;

    if (myrank == 0){
        MPI_Barrier(main_comm);
        raise(SIGKILL);
    }else{
        MPI_Barrier(main_comm);
    }
    //err_fl = false;
    flag = true;
    while (err_fl || flag) {
        err_fl = false;  
            for (j = N - 2; j >= 0; j--) {
                if(myrank < proc_num){
                    for (k = myrank; k <= j; k += proc_num){
                        A(k, N) = A(k, N) - A(k, j + 1) * X[j + 1];
                    }
                }
                //printf("!!\n");    
                MPI_Bcast(&A(j, N), 1, MPI_DOUBLE, j % proc_num, main_comm);
                //printf("!!!!\n");
                X[j] = A(j, N) / A(j, j);
                if(err_fl){
                    break;
                }
            }
        //MPI_Barrier(main_comm);
        flag = false;
        if (!err_fl)
            MPI_Barrier(main_comm);
      
    }
    //printf("!!!!!");
    saveData();
    double time1 = MPI_Wtime();

    if (myrank == 0) {
        printf("Time in seconds=%gs\n", time1 - time0);
        prt1a("X=(", X, N > 9 ? 9 : N, "...)\n");
        printf("Final_proc_num - %d\n", proc_num );
    }

    free(A);
    free(X);
    
    MPI_Barrier(main_comm);
    MPI_Finalize();
    return 0;
}

void prt1a(char *t1, double *v, int n, char *t2)
{
    int j;
    printf("%s", t1);
    for (j = 0; j < n; j++)
        printf("%.4g%s", v[j], j % 10 == 9 ? "\n" : ", ");
    printf("%s", t2);
}

void print_matrix(double* a) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N + 1; j++)
            printf("%lf ", A(i, j));
        printf("\n");
    }
    printf("\n");
}


```